In [45]:
import mlflow
import os
from mlflow import MlflowClient

import pandas as pd

# 1. Configure MLflow Tracking
mlflow.set_tracking_uri("http://localhost:5000")

# 2. Configure S3/MinIO Credentials (REQUIRED for artifact access)
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"

# # 3. Now the model can be downloaded and loaded
# model_uri = "runs:/2693624630444109a128018953c1ec7f/xgboost_churn"
# model = mlflow.pyfunc.load_model(model_uri)

# print("Model loaded successfully!")

client = MlflowClient(tracking_uri="http://localhost:5000")


In [42]:
candidate_version = client.get_model_version(
    name="model_xgboost_v0.3.2",
    version="2"
)

In [43]:
candidate_version

<ModelVersion: aliases=['champion'], creation_timestamp=1767948989860, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='XGBoost model for customer churn prediction', last_updated_timestamp=1767948989860, metrics=None, model_id=None, name='model_xgboost_v0.3.2', params=None, run_id='0e63c769c40541c7bf27b37e1d008f49', run_link='', source='runs:/0e63c769c40541c7bf27b37e1d008f49/model', status='READY', status_message=None, tags={'source_run': '0e63c769c40541c7bf27b37e1d008f49'}, user_id='', version='2'>

In [44]:
candidate_run = client.get_run("1c986928a6ce4d629587e4dc980e77a3")

In [46]:
candidate_run

<Run: data=<RunData: metrics={'accuracy_score': 0.9993194964273563,
 'example_count': 8817.0,
 'f1_score': 0.9996490407112775,
 'false_negatives': 1.0,
 'false_positives': 5.0,
 'precision_score': 0.9994152046783625,
 'recall_score': 0.9998829861923707,
 'true_negatives': 266.0,
 'true_positives': 8545.0}, params={}, tags={'mlflow.datasets': '[{"name":"dataset","hash":"317aade61b5028a6ab9ee5f60f12d495","model":"m-36e894c63979418891d1b2d57a112ba6"}]',
 'mlflow.runName': 'eval_0e63c769c40541c7bf27b37e1d008f49',
 'mlflow.source.git.commit': 'd6bdd861b82105cb0ba076b47497b1132c07a80d',
 'mlflow.source.name': '/home/mlops/Repository/aio2025-mlops-project01/model-system/src/scripts/eval.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'mlops',
 'model_uri': 'runs:/0e63c769c40541c7bf27b37e1d008f49/xgboost_churn_test_promote',
 'source_run_id': '0e63c769c40541c7bf27b37e1d008f49',
 'task': 'model_evaluation'}>, info=<RunInfo: artifact_uri='s3://mlflow/1c986928a6ce4d629587e4dc980e77a3/artifac

In [ ]:
filter_string = (
    "tags.source_run_id = '0e63c769c40541c7bf27b37e1d008f49'"
)

runs = client.search_runs(
    experiment_ids=[eval_run.info.experiment_id],
    filter_string=filter_string,
    max_results=1,
    order_by=["start_time DESC"]
)


RestException: INVALID_PARAMETER_VALUE: Invalid attribute key 'source_run_id' specified. Valid keys are '{'Run Name', 'artifact_uri', 'run_name', 'Created', 'Run name', 'status', 'user_id', 'run_id', 'created', 'end_time', 'run name', 'start_time'}'

In [62]:
runs

[]

In [51]:
eval_run = client.get_run('1c986928a6ce4d629587e4dc980e77a3')

In [53]:
eval_run.data.tags

{'mlflow.datasets': '[{"name":"dataset","hash":"317aade61b5028a6ab9ee5f60f12d495","model":"m-36e894c63979418891d1b2d57a112ba6"}]',
 'mlflow.runName': 'eval_0e63c769c40541c7bf27b37e1d008f49',
 'mlflow.source.git.commit': 'd6bdd861b82105cb0ba076b47497b1132c07a80d',
 'mlflow.source.name': '/home/mlops/Repository/aio2025-mlops-project01/model-system/src/scripts/eval.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'mlops',
 'model_uri': 'runs:/0e63c769c40541c7bf27b37e1d008f49/xgboost_churn_test_promote',
 'source_run_id': '0e63c769c40541c7bf27b37e1d008f49',
 'task': 'model_evaluation'}

In [ ]:
import pandas as pd

df = pd.read_parquet('/home/mlops/Repository/data-pipeline/churn_feature_store/churn_features/feature_repo/data/processed_churn_data.parquet')

In [ ]:
df.columns

Index(['customer_id', 'event_timestamp', 'created_timestamp', 'age', 'gender',
       'tenure_months', 'usage_frequency', 'support_calls',
       'payment_delay_days', 'subscription_type', 'contract_length',
       'total_spend', 'last_interaction_days', 'tenure_age_ratio',
       'spend_per_usage', 'support_calls_per_tenure', 'avg_monthly_spend',
       'churn_risk_score', 'churned'],
      dtype='object')

In [ ]:
df.describe()

,event_timestamp,created_timestamp,age,tenure_months,usage_frequency,support_calls,payment_delay_days,total_spend,last_interaction_days,tenure_age_ratio,spend_per_usage,support_calls_per_tenure,avg_monthly_spend,churn_risk_score,churned
count,44084,44084,44084.000000,44084.000000,44084.000000,44084.000000,44084.000000,44084.000000,44084.000000,44084.000000,44084.000000,44084.000000,44084.000000,44084.000000,44084.000000
mean,2025-11-23 20:38:37.886565120,2026-01-07 20:20:22.764388,41.532665,30.512431,15.457740,5.034979,15.034843,549.039720,15.510049,0.811772,55.808206,0.313148,43.296053,0.960860,0.969876
min,2025-10-09 20:23:21.764389,2026-01-07 20:20:22.764389,18.000000,1.000000,1.000000,0.000000,0.000000,100.000000,1.000000,0.015152,3.225806,0.000000,1.683333,0.000000,0.000000
25%,2025-11-01 04:13:23.264389120,2026-01-07 20:20:22.764389,29.000000,15.000000,8.000000,2.000000,7.000000,325.000000,8.000000,0.361111,19.411096,0.072727,10.416667,1.000000,1.000000
50%,2025-11-24 00:21:12.264389120,2026-01-07 20:20:22.764389,42.000000,31.000000,15.000000,5.000000,15.000000,549.000000,16.000000,0.718750,33.403704,0.160714,18.000000,1.000000,1.000000
75%,2025-12-16 12:02:04.764389120,2026-01-07 20:20:22.764389,54.000000,46.000000,23.000000,8.000000,23.000000,772.000000,23.000000,1.111111,61.625000,0.310345,35.500000,1.000000,1.000000
max,2026-01-07 20:18:31.764389,2026-01-07 20:20:22.764389,65.000000,60.000000,30.000000,10.000000,30.000000,1000.000000,30.000000,3.157895,500.000000,5.000000,999.000000,1.000000,1.000000
std,NaN,NaN,13.869868,17.409520,8.687956,3.155758,8.960408,259.189895,8.654741,0.581136,67.300048,0.532473,92.291100,0.148032,0.170931


In [ ]:
sample = df[['age', 'gender', 'tenure_months', 'usage_frequency', 'support_calls', 'payment_delay_days', 'subscription_type', 'contract_length', 'total_spend', 'last_interaction_days']][:5]

answer = df['churned'][:5]

In [ ]:
sample

,age,gender,tenure_months,usage_frequency,support_calls,payment_delay_days,subscription_type,contract_length,total_spend,last_interaction_days
0,30,Female,39,14.0,5,18.0,Standard,Annual,932.0,17
1,65,Female,49,1.0,10,8.0,Basic,Monthly,557.0,6
2,55,Female,14,4.0,6,18.0,Basic,Quarterly,185.0,3
3,58,Male,38,21.0,7,7.0,Standard,Monthly,396.0,29
4,23,Male,32,20.0,5,8.0,Basic,Monthly,617.0,20


In [ ]:
model.predict(sample), answer.values

(array([1, 1, 1, 1, 1]), array([1., 1., 1., 1., 1.]))